# PyRevealed: Basic Workflow Tutorial

This notebook tests the core user workflow with the tech-friendly API.

**Goal**: Smoke test the main package functionality as a new user would experience it.

## Workflow 1: Import & Setup

In [ ]:
from pyrevealed import (
    BehaviorLog,
    BehavioralAuditor,
    PreferenceEncoder,
    validate_consistency,
    compute_integrity_score,
    compute_confusion_metric,
)
import numpy as np

print("All imports successful!")

## Workflow 2: Create BehaviorLog

**Scenario**: E-commerce user purchasing 3 product categories over 5 shopping sessions.

In [ ]:
# Product categories: Electronics, Clothing, Food
prices = np.array([
    [100.0, 30.0, 10.0],   # Session 1
    [80.0, 40.0, 12.0],    # Session 2 (electronics sale)
    [120.0, 25.0, 8.0],    # Session 3 (clothing sale)
    [90.0, 35.0, 15.0],    # Session 4
    [110.0, 28.0, 11.0],   # Session 5
])

quantities = np.array([
    [0.5, 2.0, 5.0],   # Bought some of each
    [1.0, 1.0, 4.0],   # More electronics when cheap
    [0.2, 4.0, 6.0],   # More clothing when cheap
    [0.8, 2.0, 3.0],
    [0.4, 3.0, 4.0],
])

log = BehaviorLog(cost_vectors=prices, action_vectors=quantities, user_id="test_user")
print(f"Records: {log.num_records}")
print(f"Features: {log.num_features}")

## Workflow 3: Run Consistency Check (GARP)

In [ ]:
result = validate_consistency(log)
print(f"GARP consistent: {result.is_consistent}")
print(f"Violations: {result.violations}")
print(f"Computation time: {result.computation_time_ms:.2f} ms")

## Workflow 4: Compute Integrity & Confusion Metrics

In [ ]:
# Integrity score (Afriat Efficiency Index)
integrity = compute_integrity_score(log)
print(f"Integrity (AEI): {integrity.efficiency_index:.3f}")
print(f"Perfectly consistent: {integrity.is_perfectly_consistent}")

# Confusion metric (Money Pump Index)
confusion = compute_confusion_metric(log)
print(f"\nConfusion (MPI): {confusion.mpi_value:.3f}")
print(f"Cycle count: {confusion.cycle_count}")

## Workflow 5: High-Level BehavioralAuditor API

In [ ]:
auditor = BehavioralAuditor()
report = auditor.full_audit(log)

print(f"Consistent: {report.is_consistent}")
print(f"Integrity: {report.integrity_score:.3f}")
print(f"Confusion: {report.confusion_score:.3f}")

## Workflow 6: PreferenceEncoder (ML Integration)

In [ ]:
encoder = PreferenceEncoder()
encoder.fit(log)

values = encoder.extract_latent_values()
weights = encoder.extract_marginal_weights()

print(f"Latent values shape: {values.shape}")
print(f"Marginal weights shape: {weights.shape}")
print(f"\nLatent values: {values}")
print(f"Marginal weights: {weights}")

## Workflow 8: Error Handling Tests

In [ ]:
# Test negative prices - should raise an error
try:
    bad_log = BehaviorLog(
        cost_vectors=np.array([[-1.0, 2.0]]),
        action_vectors=np.array([[1.0, 1.0]])
    )
    print("ERROR: Should have raised an exception for negative prices!")
except Exception as e:
    print(f"Caught (expected): {type(e).__name__}")
    print(f"Message: {e}")

In [ ]:
# Test mismatched dimensions - should raise an error
try:
    bad_log = BehaviorLog(
        cost_vectors=np.array([[1.0, 2.0]]),
        action_vectors=np.array([[1.0, 2.0, 3.0]])
    )
    print("ERROR: Should have raised an exception for dimension mismatch!")
except Exception as e:
    print(f"Caught (expected): {type(e).__name__}")
    print(f"Message: {e}")

## Workflow 9: Edge Cases

In [ ]:
# Single observation (trivially consistent)
single = BehaviorLog(
    cost_vectors=np.array([[1.0, 2.0]]),
    action_vectors=np.array([[3.0, 1.0]])
)
result = validate_consistency(single)
print(f"Single obs consistent: {result.is_consistent}")
print(f"Expected: True (trivially consistent)")

In [ ]:
# Known violation data (WARP violation)
violation_log = BehaviorLog(
    cost_vectors=np.array([[1.0, 0.1], [0.1, 1.0]]),
    action_vectors=np.array([[1.0, 0.0], [0.0, 1.0]])
)
result = validate_consistency(violation_log)
print(f"Violation consistent: {result.is_consistent}")
print(f"Expected: False (known WARP violation)")
print(f"Violations found: {result.violations}")

## Summary

### Issues Found

1. **scipy OptimizeWarning**: When fitting `PreferenceEncoder`, scipy emits a warning about 
   an unrecognized `tol` option being passed to HiGHS solver. This is cosmetic but may confuse users.

### What Worked

- All core imports successful
- `BehaviorLog` creation with `cost_vectors`/`action_vectors` works perfectly
- `validate_consistency()` correctly identifies GARP violations
- `compute_integrity_score()` and `compute_confusion_metric()` return expected results
- `BehavioralAuditor.full_audit()` provides convenient summary
- `PreferenceEncoder.fit()` extracts features successfully
- Error handling provides clear, actionable messages with hints
- Edge cases (single observation, violations) behave correctly